In [ ]:
# ===============================================
# LSTM Sentiment Classifier (TensorFlow/Keras)
# ===============================================

import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.metrics import classification_report, confusion_matrix

# --- 0. ตั้งค่าพารามิเตอร์ ---
# ใช้ค่าเดียวกับในโมเดล PyTorch ของคุณ
EMBEDDING_DIM = 256
HIDDEN_DIM = 512
DROPOUT = 0.4
BATCH_SIZE = 64
EPOCHS = 10

# พารามิเตอร์สำหรับ Keras TextVectorization
MAX_SEQUENCE_LENGTH = 250  # กำหนดความยาวสูงสุดของประโยค (PyTorch ทำ padding ตาม batch)
MAX_TOKENS = 20000         # กำหนดขนาดของคำศัพท์ (Vocab size)

# --- 1. โหลด Dataset ---
# ! หมายเหตุ: คุณต้องอัปโหลดไฟล์ "amazon_reviews.csv" ไปยัง Colab ก่อน
try:
    df = pd.read_csv("amazon_reviews.csv")
except FileNotFoundError:
    print("="*50)
    print("🚨 ไม่พบไฟล์ 'amazon_reviews.csv'")
    print("กรุณาอัปโหลดไฟล์ไปยัง Colab (ลากไฟล์วางในแถบ 'Files' ด้านซ้าย) แล้วรันเซลล์นี้อีกครั้ง")
    print("="*50)
    # สร้าง DataFrame เปล่าเพื่อกันโค้ดแครชระหว่างรออัปโหลด
    df = pd.DataFrame(columns=["reviewText", "overall"])

if not df.empty:
    # keep only useful columns
    df = df[["reviewText", "overall"]].dropna()

    # convert ratings to binary sentiment
    df = df[df["overall"] != 3]  # remove neutral
    df["label"] = (df["overall"] >= 4).astype(int)

    # --- 2. Upsample minority class (negative = 0) ---
    # ส่วนนี้เหมือนเดิมทุกประการ เพราะใช้ pandas และ sklearn
    pos_df = df[df["label"] == 1]
    neg_df = df[df["label"] == 0]

    # upsample the negatives to match positives
    neg_df_upsampled = resample(
        neg_df,
        replace=True,         # sample with replacement
        n_samples=len(pos_df),  # match number of positive samples
        random_state=42
    )

    # combine and shuffle
    df_balanced = pd.concat([pos_df, neg_df_upsampled]).sample(frac=1, random_state=42).reset_index(drop=True)

    print("Before upsampling:")
    print(df["label"].value_counts())
    print("\nAfter upsampling:")
    print(df_balanced["label"].value_counts())

    # --- 3. Split data ---
    # แยกข้อมูลก่อน (Keras จะ .adapt() เฉพาะบน train set เพื่อป้องกัน data leakage)
    train_texts, test_texts, train_labels, test_labels = train_test_split(
        df_balanced["reviewText"], df_balanced["label"].values, # .values เพื่อให้เป็น numpy array
        test_size=0.2,
        random_state=42
    )

    # --- 4. Tokenizer และ Encoding (Keras way) ---
    # Keras ใช้ TextVectorization layer ในการจัดการ Tokenize, Build Vocab, และ Encode
    # ทั้งหมดนี้ในขั้นตอนเดียว

    vectorize_layer = layers.TextVectorization(
        max_tokens=MAX_TOKENS,             # เทียบเท่ากับ vocab_size
        output_mode='int',                 # เทียบเท่ากับ encode() ของคุณ
        output_sequence_length=MAX_SEQUENCE_LENGTH # เทียบเท่ากับ pad_sequence
    )

    # สร้าง Vocabulary (เทียบเท่ากับการวนลูปสร้าง vocab dict)
    # .adapt() จะเรียนรู้คำศัพท์จากข้อมูล training
    print("\nAdapting TextVectorization layer...")
    vectorize_layer.adapt(train_texts)

    # ดูขนาด Vocab ที่สร้างได้ (รวม <PAD> และ <UNK>)
    VOCAB_SIZE = vectorize_layer.vocabulary_size()
    print(f"Vocabulary size: {VOCAB_SIZE}")

    # --- 5. สร้าง tf.data.Dataset (Keras/TF way) ---
    # นี่คือสิ่งที่มาแทน Dataset และ DataLoader ของ PyTorch

    def vectorize_text(text, label):
        text = tf.expand_dims(text, -1) # ขยายมิติสำหรับ vectorizer
        return vectorize_layer(text), label

    # สร้าง Pipelined Datasets
    train_ds = tf.data.Dataset.from_tensor_slices((train_texts, train_labels))
    train_ds = train_ds.batch(BATCH_SIZE).map(vectorize_text).prefetch(tf.data.AUTOTUNE)

    test_ds = tf.data.Dataset.from_tensor_slices((test_texts, test_labels))
    test_ds = test_ds.batch(BATCH_SIZE).map(vectorize_text).prefetch(tf.data.AUTOTUNE)

    # --- 6. Model (Keras way) ---
    # สร้างโมเดลที่สอดคล้องกับสถาปัตยกรรม PyTorch ของคุณ

    # mask_zero=True จะบอกให้ LSTM layer ไม่สนใจ padding (เทียบเท่า padding_idx=0)
    embedding_layer = layers.Embedding(VOCAB_SIZE, EMBEDDING_DIM, mask_zero=True)

    model = keras.Sequential([
        # Input layer (จำเป็นสำหรับ TextVectorization ที่อยู่นอกโมเดล)
        layers.Input(shape=(MAX_SEQUENCE_LENGTH,), dtype="int64"),

        # 1. Embedding
        embedding_layer,

        # 2. Dropout หลัง Embedding (เหมือนใน PyTorch)
        layers.Dropout(DROPOUT),

        # 3. LSTM Layer 1 (Bidirectional)
        # PyTorch: num_layers=2, dropout=0.4 (คือ dropout *ระหว่าง* layer)
        # Keras: ต้อง stack 2 layer และใส่ Dropout layer คั่นกลาง
        layers.Bidirectional(
            layers.LSTM(HIDDEN_DIM, return_sequences=True) # return_sequences=True เพื่อส่งต่อให้ layer ถัดไป
        ),

        # Dropout *ระหว่าง* LSTM layers (เทียบเท่า dropout=0.4 ใน nn.LSTM)
        layers.Dropout(DROPOUT),

        # 4. LSTM Layer 2 (Bidirectional)
        layers.Bidirectional(
            layers.LSTM(HIDDEN_DIM, return_sequences=False) # False คือเอาแค่ state สุดท้าย
        ),

        # 5. Dropout บน Hiddent State สุดท้าย (เหมือนใน PyTorch)
        layers.Dropout(DROPOUT),

        # 6. Fully Connected (Dense) Layer
        # Keras จะรวม Sigmoid activation ไว้ใน Layer สุดท้าย
        # PyTorch: nn.Linear(hidden_dim * 2, 1) + nn.Sigmoid()
        layers.Dense(1, activation='sigmoid')
    ])

    model.summary()

    # --- 7. Training setup (Keras way) ---
    # .compile() จะรวบรวม Optimizer, Loss, และ Metrics
    model.compile(
        loss='binary_crossentropy', # เทียบเท่า nn.BCELoss
        optimizer=keras.optimizers.Adam(
            learning_rate=0.001,
            clipnorm=5.0 # เทียบเท่า clip_grad_norm_
        ),
        metrics=['accuracy']
    )

    # --- 8. Train (Keras way) ---
    # model.fit() จะจัดการ training loop และ validation loop ให้ทั้งหมด
    print("\n--- 🚀 Starting Training ---")
    history = model.fit(
        train_ds,
        epochs=EPOCHS,
        validation_data=test_ds # ประเมินผล test set ไปด้วยเลยในแต่ละ epoch
    )

    # --- 9. Evaluate (Keras way) ---
    # สามารถใช้ model.evaluate() หรือดูผลลัพธ์จาก .fit()
    print("\n--- 📊 Evaluating Model ---")
    loss, accuracy = model.evaluate(test_ds)
    print(f"\n✅ Test Accuracy (Balanced via Upsampling): {accuracy:.4f}")

    # --- 10. Evaluate with metrics (Keras way) ---
    # ใช้ model.predict() เพื่อเก็บ predictions ทั้งหมด
    all_preds_prob = model.predict(test_ds)
    all_preds = (all_preds_prob > 0.5).astype(int).squeeze()

    # เราต้องดึง labels ทั้งหมดจาก test_ds ด้วย
    all_labels = np.concatenate([labels for _, labels in test_ds], axis=0)

    # Print classification report
    print("\n📊 Classification Report:")
    print(classification_report(all_labels, all_preds, target_names=["Negative", "Positive"], digits=4))

    # Optional: show confusion matrix
    print("\n🧾 Confusion Matrix:")
    print(confusion_matrix(all_labels, all_preds))

    # --- 11. Predict new sentences ---
    # สร้างฟังก์ชัน predict ที่ใช้ vectorize_layer ที่เรา adapt ไว้
    def predict_sentiment(text):
        # Keras model คาดหวัง input เป็น batch
        # 1. ห่อ text ด้วย list -> ["...text..."]
        # 2. ส่งเข้า vectorize_layer -> ได้ Tensor รูปร่าง (1, MAX_SEQUENCE_LENGTH)
        encoded_text = vectorize_layer([text])

        # 3. ส่งเข้า model.predict
        prediction = model.predict(encoded_text, verbose=0) # verbose=0 เพื่อปิด loading bar
        pred_prob = prediction[0][0] # ได้ค่า probability (เช่น 0.98)

        return "Positive" if pred_prob > 0.5 else "Negative"

    # --- 12. Try multiple examples ---
    positive_texts = [
        "I really love this product!",
        "Absolutely fantastic, exceeded my expectations.",
        "This is the best purchase I've made this year.",
        "High quality and works perfectly.",
        "I am very satisfied with this item.",
        "Excellent product, would buy again.",
        "Totally worth the money, highly recommend.",
        "Amazing! Fast shipping and great quality.",
        "Very happy with this, five stars!",
        "This product is incredible, love it!"
    ]

    negative_texts = [
        "Terrible quality, very disappointed.",
        "Broke after one use, waste of money.",
        "Do not buy this, completely useless.",
        "Poorly made and arrived late.",
        "Not as described, extremely unhappy.",
        "This product is awful, avoid it.",
        "Very disappointed, will never buy again.",
        "Cheap material, doesn't work at all.",
        "Horrible experience, one star.",
        "Regret buying this, total waste."
    ]

    print("\n--- Positive Sentiment Predictions ---")
    for text in positive_texts:
        sentiment = predict_sentiment(text)
        print(f"'{text}' -> {sentiment}")

    print("\n--- Negative Sentiment Predictions ---")
    for text in negative_texts:
        sentiment = predict_sentiment(text)
        print(f"'{text}' -> {sentiment}")

Before upsampling:
label
1    4448
0     324
Name: count, dtype: int64

After upsampling:
label
0    4448
1    4448
Name: count, dtype: int64

Adapting TextVectorization layer...
Vocabulary size: 9719


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 250, 256)       │     2,488,064 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 250, 256)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 250, 1024)      │     3,149,824 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 250, 1024)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, 1024)           │     6,295,552 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │         1,025 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,934,465 (45.53 MB)

 Trainable params: 11,934,465 (45.53 MB)

 Non-trainable params: 0 (0.00 B)


--- 🚀 Starting Training ---
Epoch 1/10
112/112 ━━━━━━━━━━━━━━━━━━━━ 27s 173ms/step - accuracy: 0.8020 - loss: 0.4682 - val_accuracy: 0.9624 - val_loss: 0.1150
Epoch 2/10
112/112 ━━━━━━━━━━━━━━━━━━━━ 19s 172ms/step - accuracy: 0.9707 - loss: 0.0904 - val_accuracy: 0.9685 - val_loss: 0.1166
Epoch 3/10
112/112 ━━━━━━━━━━━━━━━━━━━━ 19s 173ms/step - accuracy: 0.9625 - loss: 0.1303 - val_accuracy: 0.9820 - val_loss: 0.0943
Epoch 4/10
112/112 ━━━━━━━━━━━━━━━━━━━━ 19s 170ms/step - accuracy: 0.9924 - loss: 0.0410 - val_accuracy: 0.9792 - val_loss: 0.0555
Epoch 5/10
112/112 ━━━━━━━━━━━━━━━━━━━━ 19s 169ms/step - accuracy: 0.9930 - loss: 0.0286 - val_accuracy: 0.9871 - val_loss: 0.0364
Epoch 6/10
112/112 ━━━━━━━━━━━━━━━━━━━━ 19s 169ms/step - accuracy: 0.9953 - loss: 0.0194 - val_accuracy: 0.9893 - val_loss: 0.0319
Epoch 7/10
112/112 ━━━━━━━━━━━━━━━━━━━━ 19s 171ms/step - accuracy: 0.9979 - loss: 0.0084 - val_accuracy: 0.9803 - val_loss: 0.0870
Epoch 8/10
112/112 ━━━━━━━━━━━━━━━━━━━━ 19s 172ms/step